In [15]:
import numpy as np
from cellpose import utils
import cellpose as cp
import cv2
from skimage import morphology, measure
from skimage.filters import threshold_multiotsu
import skimage
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import re
from tqdm import tqdm

In [5]:
def multi_otsu_mask(img, classes=3, show_segmentation=False, savepath=None, 
                    class_level=None, binary_close_size=40, binary_dilate_size=20,
                    small_object_size=6400):
    
    # takes as input a single-channel image.
    # performs a rough (generous) segmentation of the DRG image 
    # using the multiotsu method in skimage (2 or 3 bins).
    # identifies the brightest bin as the cell body level. If this is not true, specify class_level.
    
    # if show_segmentation is True, plots a step-by-step preview of the mask manipulation for troubleshooting.
    # binary_close_size and binary_dilate_size can be changed to specify 
    # the number of pixels to use for the binary_closing and binary_dilation steps.
    # small_object_size specifies the maximum size of speckles to be removed in the remove_small_objects step.
    
    # note that this method is slow (~1min) for large images (~4000x4000px) + 3 bins.
    # otsu thresholding is exponential in runtime, so be careful going above 3 bins.    
    # returns the segmented mask and saves it if savepath is specified
    
    if classes >= 3:
        thresholds = threshold_multiotsu(img, classes=classes)
    if classes == 2:
        #otsu implementation in cv2 is faster
        ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thresholds = [ret]
        
    regions = np.digitize(img, bins=thresholds)
    
    if class_level is None:
        img_mask = regions==(classes-1)
    else:
        img_mask = regions==(class_level)
    
    # close holes
    kernel_close= morphology.disk(binary_close_size)
    closed_mask = morphology.binary_closing(img_mask, footprint=kernel_close)
    # clean mask
    cleaned_mask = morphology.remove_small_objects(closed_mask, min_size=small_object_size, connectivity=1)
    # dilate mask
    kernel_open = morphology.disk(binary_dilate_size)
    final_mask = morphology.binary_dilation(cleaned_mask, footprint=kernel_open)
    
    if show_segmentation:
        fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 5))

        # original image.
        ax[0,0].imshow(img, cmap='gray')
        ax[0,0].set_title('Original')
        ax[0,0].axis('off')

        # plot histogram and thresholds
        ax[0,1].hist(img.ravel(), bins=255)
        ax[0,1].set_title('Histogram')
        for thresh in thresholds:
            ax[0,1].axvline(thresh, color='r')

        # plot otsu segmentation
        ax[0,2].imshow(regions, cmap='jet')
        ax[0,2].set_title('Otsu = '+str(classes))
        ax[0,2].axis('off')

        # plot mask after binary close operation
        ax[1,0].imshow(closed_mask)
        ax[1,0].set_title('closed mask')
        ax[1,0].axis('off')

        # plot mask after removing speckles
        ax[1,1].imshow(cleaned_mask)
        ax[1,1].set_title('cleaned mask')
        ax[1,1].axis('off')

        # plot mask after binary dilation
        ax[1,2].imshow(final_mask)
        ax[1,2].set_title('final mask')
        ax[1,2].axis('off')

        plt.subplots_adjust()

        plt.show()
    if savepath is not None:
        skimage.io.imsave(savepath + '_rough.png', skimage.img_as_ubyte(final_mask))
    return final_mask

def measure_masks_in_cropped_region(dat, crop_region, savedir=None):
    # assumes input dat is a loaded seg.npy file
    # cp_masks is the list of masks from cellpose
    # crop_region is the rough mask from otsu thresholding
    # if savedir is not None, save the cropped masks in the directory
    # returns dataframe of properties of objects in the cropped region of the mask.
   
    cp_masks = dat['masks']
    crop = cp_masks * crop_region
    
    red_features = ['label', 'area','eccentricity','solidity', 'feret_diameter_max','perimeter','intensity_mean','intensity_max','intensity_min']
    green_features = ['intensity_mean','intensity_max','intensity_min']
    # on channel 0 (red)
    props_red = measure.regionprops_table(crop, dat['img'][0], properties=red_features)
    df = pd.DataFrame(props_red)
    df.rename(columns={'intensity_mean':'Red_mean', 'intensity_max':'Red_max','intensity_min':'Red_min'}, inplace=True)
    # on channel 1 (green)
    props_green = measure.regionprops_table(crop, dat['img'][1], properties=green_features)
    df['Green_mean'] = props_green['intensity_mean']
    df['Green_max'] = props_green['intensity_max']
    df['Green_min'] = props_green['intensity_min']
    
    if savedir is not None:
        cp.io.save_masks(dat['img'][0], crop, dat['flows'], dat['filename'], savedir=savedir, save_txt=False)
        
    return df

In [ ]:
save_dir = "placeholder"
folder = "placeholder"

df = pd.DataFrame()
os.chdir(folder)
files = glob.glob('*_seg.npy')

for i in tqdm(range(len(files))):
    file = files[i]
    dat = np.load(file, allow_pickle=True).item()

    # extract some information from the title
    pattern = re.compile(r"""^animal(?P<Animal>[0-9]{2})-slide(?P<Slide>[0-9])-(?P<Type>.*)_(?P<SliceNumber>[0-9]{4})""", re.VERBOSE)
    match = pattern.match(file)
    animal = int(match.group('Animal'))
    slide = int(match.group('Slide'))
    antibody = match.group('Type')
    imgslice = int(match.group('SliceNumber'))

    otsu_mask = multi_otsu_mask(dat['img'][0], classes=3, savepath=folder+"/otsu_masks/"+file[:-8]+".png")

    df_img = measure_masks_in_cropped_region(dat, otsu_mask, savedir=folder+"/otsu_masks/")
    df_img['Animal'] = animal
    df_img['Slide'] = slide
    df_img['Type'] = antibody
    df_img['ImgSlice'] = imgslice

    #save individual files in case of broken loop
    df_img.to_csv(save_dir+"/individual_csv_otsu/"+file[:-8]+".csv")        

    df = pd.concat([df, df_img])

# save output to pickle and csv
df.to_pickle(save_dir+"/summary_dataframe_otsu.pkl")
df.to_csv(save_dir+"/summary_dataframe_otsu.csv")